In [1]:
!pip3 install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [3]:
import pandas as pd
import tensorflow as tf
from modules.utils import get_dataset
from modules.evaluate import Evaluate

2023-02-21 07:20:55.060679: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/Sampling'

In [5]:
RESULT_EVALUATE_PATH = os.path.join(CURRENT_PATH, "results", "evaluate", "facal_loss", "EfficientNetB0_None")
RESULT_EVALUATE_PATH

'/home/jovyan/ChestXray-14/Sampling/results/evaluate/facal_loss/EfficientNetB0_None'

In [6]:
from pathlib import Path
Path(RESULT_EVALUATE_PATH).mkdir(parents=True, exist_ok=True)

In [7]:
class Dataset:
    INPUT_PATH = "/home/jovyan/ChestXray-14/dataset/ChestXray NIH"
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

In [8]:
test_dataset = Dataset().get_test()

2023-02-21 07:20:57.127073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 07:20:58.901099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31727 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:ca:00.0, compute capability: 8.0


In [9]:
MODEL_PATH = '/home/jovyan/ChestXray-14/Sampling/results/models/FacalLoss/EfficientNetB0_None.h5'
best_model = Evaluate(MODEL_PATH)
best_model.get_best_threshold(
    test_dataset=test_dataset,
    save_best_thresholds=f"{RESULT_EVALUATE_PATH}/best_thresholds.csv",
    save_200_thresholds=f"{RESULT_EVALUATE_PATH}/f1_per_thresholds.csv"
)

with best_model:
    f1_each_class = best_model.get_f1_scores(test_dataset)
    df = pd.DataFrame(f1_each_class)
    df.to_csv(f"{RESULT_EVALUATE_PATH}/f1_scores.csv", index=False)

    precision_each_class = best_model.get_precision_scores(test_dataset, new_calculate=False)
    pd.DataFrame(precision_each_class)\
        .to_csv(f"{RESULT_EVALUATE_PATH}/precision.csv", index=False)

    recall_each_class = best_model.get_recall_scores(test_dataset, new_calculate=False)
    pd.DataFrame(recall_each_class)\
        .to_csv(f"{RESULT_EVALUATE_PATH}/recall.csv", index=False)

    print(df)

2023-02-21 07:21:16.309974: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-21 07:21:17.625029: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-21 07:21:17.626803: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-21 07:21:17.626859: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-21 07:21:17.627910: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-21 07:21:17.628043: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


      7/Unknown - 10s 27ms/step

2023-02-21 07:21:22.259508: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


765/765 [==============================] - 35s 33ms/step
['No Finding', 'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']
/home/jovyan/ChestXray-14/Sampling/results/evaluate/facal_loss/EfficientNetB0_None/best_thresholds.csv was success!
/home/jovyan/ChestXray-14/Sampling/results/evaluate/facal_loss/EfficientNetB0_None/f1_per_thresholds.csv was success!
Doing ...!
765/765 [==============================] - 22s 29ms/step
   No Finding  Atelectasis  Consolidation  Infiltration  Pneumothorax  \
0    0.407764     0.428323       0.225931      0.511919      0.322133   

      Edema  Emphysema  Fibrosis  Effusion  Pneumonia  Pleural_Thickening  \
0  0.216495   0.164651  0.093981  0.531902   0.066055            0.183575   

   Cardiomegaly    Nodule      Mass   Hernia  
0      0.368087  0.233749  0.273158  0.04878  
Done!
